<a href="https://colab.research.google.com/github/PritiRangnekar/CS221FinalProjectEllo/blob/main/DistilGPT2Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import torch
from torch.utils.data import Dataset
import re
!pip install transformers
import transformers
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
# Use Google Collab's GPU
if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    print('No GPU available')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 23.6 MB/s 
     |████████████████████████████████| 596 kB 51.5 MB/s 
     |████████████████████████████████| 6.6 MB 53.7 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
# Set our random seed for running torch
torch.manual_seed(42)
# Download the tokenizer for a distilled GPT-2 from the huggingface library 
# (note that the full GPT-2 exceeded our free runtime/storage to train on google collab).
# Also add tokens for the beginning and the end of the sequence and a special token for padding. 
tokenizer = AutoTokenizer.from_pretrained("distilgpt2", bos_token='<startoftext>', eos_token='<endoftext>', pad_token='<pad>')
# Download the pre-trained distilled model from hugginface and transfer it to the GPU
model = AutoModelForCausalLM.from_pretrained("distilgpt2").cuda()
# Resize the token embeddings because we've just added 3 new tokens 
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/336M [00:00<?, ?B/s]

Embedding(50260, 768)

In [3]:
# import our csvs into dataframes
train_file_path = 'CONTEXT_b1Train'
eval_file_path = 'CONTEXT_b1Val'
test_file_path = 'CONTEXT_b1Test'

def read_data(file_path): 
  df = pd.read_csv(file_path, header=None)
  df.columns = ['Text', 'Commentary', 'ISBN', 'Page']
  x, y, isbn, page = df['Text'].tolist(), df['Commentary'].tolist(), df['ISBN'].tolist(), df['Page'].tolist()
  x.pop(0)
  y.pop(0)
  isbn.pop(0)
  page.pop(0)
  return x, y, isbn, page

x_train, y_train, isbn_train, page_train = read_data(test_file_path)
x_eval, y_eval, isbn_eval, page_eval = read_data(eval_file_path)
x_test, y_test, isbn_test, page_test = read_data(test_file_path)

# Use the HuggingFace dataset class for our training data 
class CommentaryDataset(Dataset):
  def __init__(self, txt_list, com_list, tokenizer, max_length):
    self.input_ids = []
    self.attn_masks = []
    self.labels = []
    for txt, commentary in zip(txt_list, com_list):
      # our training examples should include the page and commentary, with commentary as the prompt to train on.
      prep_txt = f'<startoftext>Page: {txt}\nCommentary: {commentary}<endoftext>'
      # tokenize our examples using the GPT-2 tokenizer (specified later), while padding them to our max length
      encodings_dict = tokenizer(prep_txt, truncation=True, 
                                max_length=max_length, padding='max_length')
      input_ids = torch.tensor(encodings_dict['input_ids'])    
      self.input_ids.append(input_ids)
      mask = torch.tensor(encodings_dict['attention_mask'])
      self.attn_masks.append(mask)
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]

train_dataset = CommentaryDataset(x_train, y_train, tokenizer, max_length=512)

In [8]:
# define our training arguments (we modified these slightly through experimentation)
training_args = TrainingArguments(output_dir='/Users/eddiedilworth/Downloads/221_Final_Project_results',
                                  num_train_epochs=10,
                                  logging_steps=10,
                                  save_strategy="epoch",
                                  weight_decay = 0.01,
                                  per_device_train_batch_size=2,  
)
Trainer(model=model, args=training_args,  
                  train_dataset=train_dataset, 
                  data_collator=lambda data: 
              {'input_ids': torch.stack([f[0] for f in data]),       
               'attention_mask': torch.stack([f[1] for f in data]),
               'labels': torch.stack([f[0] for f in data])}).train()
               

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 84
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 420


Step,Training Loss
10,0.664200
20,0.636000
30,0.541600
40,0.493000
50,0.649000
60,0.420000
70,0.466100
80,0.485400
90,0.462900
100,0.396900


Saving model checkpoint to /Users/eddiedilworth/Downloads/cs124/221_Final_Project_results/checkpoint-42
Configuration saved in /Users/eddiedilworth/Downloads/cs124/221_Final_Project_results/checkpoint-42/config.json
Model weights saved in /Users/eddiedilworth/Downloads/cs124/221_Final_Project_results/checkpoint-42/pytorch_model.bin
Saving model checkpoint to /Users/eddiedilworth/Downloads/cs124/221_Final_Project_results/checkpoint-84
Configuration saved in /Users/eddiedilworth/Downloads/cs124/221_Final_Project_results/checkpoint-84/config.json
Model weights saved in /Users/eddiedilworth/Downloads/cs124/221_Final_Project_results/checkpoint-84/pytorch_model.bin
Saving model checkpoint to /Users/eddiedilworth/Downloads/cs124/221_Final_Project_results/checkpoint-126
Configuration saved in /Users/eddiedilworth/Downloads/cs124/221_Final_Project_results/checkpoint-126/config.json
Model weights saved in /Users/eddiedilworth/Downloads/cs124/221_Final_Project_results/checkpoint-126/pytorch_model

TrainOutput(global_step=420, training_loss=0.38365690821693055, metrics={'train_runtime': 127.832, 'train_samples_per_second': 6.571, 'train_steps_per_second': 3.286, 'total_flos': 109744635248640.0, 'train_loss': 0.38365690821693055, 'epoch': 10.0})

In [11]:
# Define a function that prompts our output commentary
def create_outputs(testingtext, ISBN, Page): 
  commentaries = [] 
  total = []
  for text in testingtext: 
    # our prompt ends on "Commentary:", allowing the model to generate the next words. 
    prompt = f'<startoftext>Page: {text}\nCommentary:'
    generated = tokenizer(f'{prompt}', return_tensors= 'pt').input_ids.cuda()
    # We experimented a bit with temperature and top_p, and generally found that these standard defaults avoided loops, without too much
    # unexpected commentary. They would be parameters to explore more in the future. 
    sample_outputs = model.generate(generated, do_sample=False, top_k=50, max_length=200, top_p=.90, temperature=0.1, num_return_sequences=0)
    predicted_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
    # retrieve just the commentary with a regex
    predicted_commentary = re.findall("\nCommentary:(.*)", predicted_text)[-1]
    total.append(predicted_text)
    commentaries.append(predicted_commentary)
  return total, commentaries

# generate eval commentary
total, eval_commentaries = create_outputs(x_eval, isbn_eval, page_test)
eval_csv = pd.DataFrame({'Text': x_eval, 'Commentary': eval_commentaries, 'ISBN': isbn_eval, 'Page': page_eval})
eval_csv.to_csv('GPTCommentaries_batchOneVal.csv')

# only generate test commentary after we have optimized all our parameters.
#total, test_commentaries = create_outputs(x_test, isbn_test, page_test)
#test_csv = pd.DataFrame({'Text': x_test, 'Commentary': test_commentaries, 'ISBN': isbn_test, 'Page': page_test})
#test_csv.to_csv('GPTCommentaries_batchOneTest.csv')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene